In [2]:

import os
from typing import List, Dict, Tuple, Optional, Any

import numpy as np
import tensorflow as tf

from waymo_open_dataset import dataset_pb2
from waymo_open_dataset.protos import scenario_pb2
from waymo_open_dataset.protos import compressed_lidar_pb2
from waymo_open_dataset.utils import womd_lidar_utils
     

In [ ]:
def _load_scenario_data(tfrecord_file: str) -> scenario_pb2.Scenario:
  """Load a scenario proto from a tfrecord dataset file."""
  dataset = tf.data.TFRecordDataset(tfrecord_file, compression_type='')
  data = next(iter(dataset))
  return scenario_pb2.Scenario.FromString(data.numpy())

WOMD_FILE = 'data/segment-10017090168044687777_6380_000_6400_000_with_camera_labels.tfrecord'
womd_original_scenario = _load_scenario_data(WOMD_FILE)
print(womd_original_scenario.scenario_id)

2025-10-03 17:55:28.232281: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-10-03 17:55:28.260447: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


DecodeError: Error parsing message

In [4]:
import zlib
import numpy as np
import tensorflow as tf

from waymo_open_dataset import dataset_pb2 as _dataset_pb2
from waymo_open_dataset.protos import scenario_pb2
from waymo_open_dataset.utils import frame_utils

In [8]:

def scenario_to_first_frame(path):
    dataset = tf.data.TFRecordDataset(path, compression_type='')
    raw = next(iter(dataset))
    scenario = scenario_pb2.Scenario()
    dir(scenario)
    scenario.ParseFromString(bytes(raw.numpy()))
    
    # LiDAR data lives in scenario.lidar_data (list, one per timestamp).
    # Take the first lidar_data entry:
    lidar_record = scenario.lidar_data[0].lidar_points_compressed

    # Decompress
    frame_bytes = zlib.decompress(lidar_record)
    frame = _dataset_pb2.Frame()
    frame.ParseFromString(frame_bytes)
    return frame

In [9]:
frame = scenario_to_first_frame(WOMD_FILE)

DecodeError: Error parsing message

In [14]:
type(raw)

tensorflow.python.framework.ops.EagerTensor

In [20]:
dataset = tf.data.TFRecordDataset('data/training_20s.tfrecord-00000-of-01000', compression_type='')
scenario_data = []
for data in dataset:
    proto_string = data.numpy()
    proto = scenario_pb2.Scenario()
    proto.ParseFromString(proto_string)
    scenario_data.append(proto)


In [28]:
dir(scenario_data[0].timestamp_micros)

AttributeError: timestamp_micros

In [26]:
type(scenario_data[0])

waymo_open_dataset.protos.scenario_pb2.Scenario

In [29]:
womd_lidar_utils.augment_womd_scenario_with_lidar_points

<function waymo_open_dataset.utils.womd_lidar_utils.augment_womd_scenario_with_lidar_points(scenario: waymo_open_dataset.protos.scenario_pb2.Scenario, lidar_data: waymo_open_dataset.protos.scenario_pb2.Scenario) -> waymo_open_dataset.protos.scenario_pb2.Scenario>

In [1]:
#@title Initial setup
from typing import Optional
import warnings
# Disable annoying warnings from PyArrow using under the hood.
warnings.simplefilter(action='ignore', category=FutureWarning)


import tensorflow as tf
import dask.dataframe as dd
from waymo_open_dataset import v2

paths = tf.io.gfile.glob(f'/home/ape/repos/road_quality/data/10017090168044687777_6380_000_6400_000.parquet')
data = dd.read_parquet(paths)


2025-10-03 18:46:36.870377: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-03 18:46:37.475352: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
data.head()

,key.segment_context_name,key.frame_timestamp_micros,key.laser_name,[LiDARComponent].range_image_return1.values,[LiDARComponent].range_image_return1.shape,[LiDARComponent].range_image_return2.values,[LiDARComponent].range_image_return2.shape
index,,,,,,,
10017090168044687777_6380_000_6400_000;1550083467346370,10017090168044687777_6380_000_6400_000,1550083467346370,1,"[20.885931, 0.19042969, 1.493107, -1.0, 18.936...","[64, 2650, 4]","[-1.0, -1.0, -1.0, -1.0, 21.003038, 0.14941406...","[64, 2650, 4]"
10017090168044687777_6380_000_6400_000;1550083467346370,10017090168044687777_6380_000_6400_000,1550083467346370,2,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[200, 600, 4]","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[200, 600, 4]"
10017090168044687777_6380_000_6400_000;1550083467346370,10017090168044687777_6380_000_6400_000,1550083467346370,3,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[200, 600, 4]","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[200, 600, 4]"
10017090168044687777_6380_000_6400_000;1550083467346370,10017090168044687777_6380_000_6400_000,1550083467346370,4,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[200, 600, 4]","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[200, 600, 4]"
10017090168044687777_6380_000_6400_000;1550083467346370,10017090168044687777_6380_000_6400_000,1550083467346370,5,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[200, 600, 4]","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[200, 600, 4]"


In [3]:
data.columns

Index(['key.segment_context_name', 'key.frame_timestamp_micros',
       'key.laser_name', '[LiDARComponent].range_image_return1.values',
       '[LiDARComponent].range_image_return1.shape',
       '[LiDARComponent].range_image_return2.values',
       '[LiDARComponent].range_image_return2.shape'],
      dtype='object')

In [2]:
#@title Initial setup
from typing import Optional
import warnings
# Disable annoying warnings from PyArrow using under the hood.
warnings.simplefilter(action='ignore', category=FutureWarning)


import tensorflow as tf
import dask.dataframe as dd
from waymo_open_dataset import v2


# Path to the directory with all components
dataset_dir = '/home/ape/repos/road_quality/data/v2'

context_name = '10017090168044687777_6380_000_6400_000'

def read(tag: str) -> dd.DataFrame:
  """Creates a Dask DataFrame for the component specified by its tag."""
  paths = tf.io.gfile.glob(f'{dataset_dir}/{tag}/{context_name}.parquet')
  return dd.read_parquet(paths)

In [3]:
# @title Basic Example (Camera images with labels)

# Lazily read camera images and boxes 
cam_image_df = read('camera_image')
cam_box_df = read('camera_box')

# Combine DataFrame for individual components into a single DataFrame.

# Camera cam_box_df will be grouped, so each row will have a camera image
# and all associated boxes.
image_w_box_df = v2.merge(cam_image_df, cam_box_df, right_group=True)

# Show raw data
image_w_box_df.head()

# Example how to access data fields via v2 object-oriented API
print(f'Available {image_w_box_df.shape[0].compute()} rows:')
for i, (_, r) in enumerate(image_w_box_df.iterrows()):
  # Create component dataclasses for the raw data
  cam_image = v2.CameraImageComponent.from_dict(r)
  cam_box = v2.CameraBoxComponent.from_dict(r)
  print(
      f'context_name: {cam_image.key.segment_context_name}'
      f' ts: {cam_image.key.frame_timestamp_micros}'
      f' camera_name: {cam_image.key.camera_name}'
      f' image size: {len(cam_image.image)} bytes.'
      f' Has {len(cam_box.key.camera_object_id)} camera labels:'
  )

  for j, (object_id, x, y) in enumerate(zip(
      cam_box.key.camera_object_id, cam_box.box.center.x, cam_box.box.center.y
  )):
    print(f'\tid: {object_id},  center: ({x:.1f}, {y:.1f}) px')
    if j > 2:
      print('\t...')
      break
  if i > 2:
    print('...')
    break

Available 685 rows:
context_name: 10017090168044687777_6380_000_6400_000 ts: 1550083467346370 camera_name: 1 image size: 412699 bytes. Has 5 camera labels:
	id: 12ba81ee-9cee-4134-94ca-df9ed01e0ff1,  center: (1466.6, 492.6) px
	id: 2ab9ff55-a510-435b-a729-884444e67249,  center: (1109.1, 536.9) px
	id: 470e3826-ecd2-425c-80ca-33f3d8c876a6,  center: (1181.7, 519.2) px
	id: 98fcf17a-d19e-4c3b-bbc0-54ae2d1fcc15,  center: (1006.1, 487.0) px
	...
context_name: 10017090168044687777_6380_000_6400_000 ts: 1550083467346370 camera_name: 2 image size: 389779 bytes. Has 2 camera labels:
	id: 63c84d73-3c52-4c22-8a14-55a797622e69,  center: (1460.9, 672.6) px
	id: df9d5e7d-c686-4425-a227-d0299043ade8,  center: (1418.6, 752.2) px
context_name: 10017090168044687777_6380_000_6400_000 ts: 1550083467446163 camera_name: 1 image size: 413054 bytes. Has 5 camera labels:
	id: 12ba81ee-9cee-4134-94ca-df9ed01e0ff1,  center: (1496.9, 485.1) px
	id: 2ab9ff55-a510-435b-a729-884444e67249,  center: (1131.8, 535.0) px

In [4]:

# Lazily read lidar boxes
lidar_box_df = read('lidar_box')
# Group all available boxes per object
lidar_box_df = (
    lidar_box_df.groupby(['key.segment_context_name', 'key.laser_object_id'])
    .agg(list)
    .reset_index()
)


# Read a single row, which contains data for all available frames.
_, row = next(iter(lidar_box_df.iterrows()))

# Create component object
lidar_box = v2.LiDARBoxComponent.from_dict(row)
print(
    f'Object {lidar_box.key.laser_object_id=} seen on'
    f' {len(lidar_box.key.frame_timestamp_micros)} frames'
)
print('Its trajectory across frames:')
print(f'\t{lidar_box.box.center.x=}')
print(f'\t{lidar_box.box.center.y=}')
print(f'\t{lidar_box.box.center.z=}')

Object lidar_box.key.laser_object_id='0QXtLAoMcF26x6k0m-7gVQ' seen on 198 frames
Its trajectory across frames:
	lidar_box.box.center.x=[31.960473150901407, 31.398161404554912, 30.771275060649486, 30.154010823531507, 29.546898993095738, 28.93718751125334, 28.3313341367425, 27.73340217996747, 27.144020457655415, 26.551439169952346, 25.96802233992412, 25.405489601466797, 24.843880380552946, 24.277074372880634, 23.713582923485774, 23.165448888217497, 22.614864031012075, 22.0539553172448, 21.492416694230087, 20.940051722913267, 20.392317597087413, 19.8525902326096, 19.3249809029885, 18.801231114375923, 18.268838220510133, 17.73669772888752, 17.222987764689833, 16.71800649605666, 16.209799339130313, 15.698476130384961, 15.189663256075164, 14.69183267696667, 14.200386668905594, 13.704296045105366, 13.207701320936621, 12.72017775356312, 12.24114945589099, 11.768598933188969, 11.294816419192557, 10.82444727476377, 10.362465998811103, 9.903456855454124, 9.452072930530903, 9.00614593535397, 8.562

In [5]:
#@title Sensor data with both lidar and camera boxes

# Lazily read DataFrames for all components.
association_df = read('camera_to_lidar_box_association')
cam_box_df = read('camera_box')
cam_img_df = read('camera_image')
lidar_box_df = read('lidar_box')
lidar_df = read('lidar')

# Join all DataFrames using matching columns
cam_image_w_box_df = v2.merge(cam_box_df, cam_img_df)
cam_obj_df = v2.merge(association_df, cam_image_w_box_df)
obj_df = v2.merge(cam_obj_df, lidar_box_df)
# Group lidar sensors (left), group labels and camera images (right) and join.
df = v2.merge(lidar_df, obj_df, left_group=True, right_group=True)

# # Read a single row, which contain data for all data for a single frame.
# _, row = next(iter(df.iterrows()))
# # Create all component objects
# camera_image = v2.CameraImageComponent.from_dict(row)
# lidar = v2.LiDARComponent.from_dict(row)
# camera_box = v2.CameraBoxComponent.from_dict(row)
# lidar_box = v2.LiDARBoxComponent.from_dict(row)

# print(
#     f'Found {len(lidar_box.key.laser_object_id)} objects on'
#     f' {lidar.key.segment_context_name=} {lidar.key.frame_timestamp_micros=}'
# )
# for laser_object_id, camera_object_id, camera_name in zip(
#     lidar_box.key.laser_object_id,
#     camera_box.key.camera_object_id,
#     camera_image.key.camera_name,
# ):
#   print(f'\t{laser_object_id=} {camera_object_id=} {camera_name=}')

In [13]:
lidar_df['[LiDARComponent].range_image_return1.values'].head()[0].shape

(678400,)

In [18]:
_, row = next(iter(lidar_df.iterrows()))

lidar = v2.LiDARComponent.from_dict(row)


In [20]:
dir(lidar.range_image_return1)

['__annotations__',
 '__class__',
 '__dataclass_fields__',
 '__dataclass_params__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__match_args__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'shape',
 'tensor',
 'values']

In [18]:
# @title Sensor data with lidar and camera boxes if available

# Lazily read DataFrames for all components.
association_df = read('camera_to_lidar_box_association')
cam_box_df = read('camera_box')
cam_img_df = read('camera_image')
lidar_box_df = read('lidar_box')
lidar_df = read('lidar')

# Join all DataFrames using matching columns
cam_image_w_box_df = v2.merge(cam_box_df, cam_img_df)
cam_obj_df = v2.merge(association_df, cam_image_w_box_df)
# In this example camera box labels are optional, so we set left_nullable=True.
obj_df = v2.merge(cam_obj_df, lidar_box_df, left_nullable=True)
# Group lidar sensors (left), group labels and camera images (right) and join.
df = v2.merge(lidar_df, obj_df, left_group=True, right_group=True)

# Read a single row, which contain data for all data for a single frame.
_, row = next(iter(df.iterrows()))
# Create all component objects
camera_image = v2.CameraImageComponent.from_dict(row)
lidar = v2.LiDARComponent.from_dict(row)
camera_box = v2.CameraBoxComponent.from_dict(row)
lidar_box = v2.LiDARBoxComponent.from_dict(row)

print(
    f'Found {len(lidar_box.key.laser_object_id)} objects on'
    f' {lidar.key.segment_context_name=} {lidar.key.frame_timestamp_micros=}'
)
for laser_object_id, camera_object_id, camera_name in zip(
    lidar_box.key.laser_object_id,
    camera_box.key.camera_object_id,
    camera_image.key.camera_name,
):
  print(f'\t{laser_object_id=} {camera_object_id=} {camera_name=}')

: 